In [ ]:
# |default_exp sample_manager
# You need this at the top of every notebook you want turned into a module, the name your provide will determine the module name

# Libraries


In [ ]:
# |export
# That export there, it makes sure this code goes into the module.

# standard libs
import os
import re

# Common to template
# add into settings.ini, requirements, package name is python-dotenv, for conda build ensure `conda config --add channels conda-forge`
import dotenv  # for loading config from .env files, https://pypi.org/project/python-dotenv/
import envyaml  # Allows to loads env vars into a yaml file, https://github.com/thesimj/envyaml
import fastcore  # To add functionality related to nbdev development, https://github.com/fastai/fastcore/
from fastcore import (
    test,
)
from fastcore.script import (
    call_parse,
)  # for @call_parse, https://fastcore.fast.ai/script
import json  # for nicely printing json and yaml
from fastcore import test
from fastcore.script import call_parse
from ssi_analysis_utility import (
    core,
)
from pathlib import Path #to be able write :Path in cli function

# Project specific libraries


In [ ]:
# To change working directory when running notebook only. DO NOT EXPORT

os.chdir(core.PROJECT_DIR)

# Sample Manager

The Sample Manager module provides tools for managing and initializing sample data from Illumina and Nanopore sequencing. It includes two main classes: `sample_data` for individual sample attributes and `input_manager` for creating and handling multiple samples efficiently.

---

## `sample_data` Class

The `sample_data` class is used to represent and store essential information for the samples. This class creates an object with the following attributes:

- **sample_name**: Identifier for the sample, e.g unique name or code.
- **assembly_file**: Path to the genome assembly file in FASTA format, e.g., `"/path/to/genome_assembly.fasta"`.
- **Illumina_read_files**: Paths to paired-end sequencing files (R1 and R2) from Illumina sequencing, formatted as a comma-separated string, e.g., `"/path/to/R1.fastq.gz,/path/to/R2.fastq.gz"`.
- **Nanopore_read_file**: Path to a file containing long-read sequencing data in FASTQ format, e.g., `"/path/to/Nanopore_read_data.fastq.gz"`.
- **metadata**: A dictionary containing metadata loaded from a TSV file, including `sample_name` and paths to data files.
- **input_folder**: Optional path used when loading multiple samples at once with the `input_manager` class (see below).

---

## `input_manager` Class

The `input_manager` class is designed to initiate `sample_data` class instances. It provides three different ways to initialize samples:

1. **Single Sample Initialization**:
    - Creates a single `sample_data` instance by directly providing the sample name and paths to the read/assembly files.

2. **Folder-Based Initialization**:
    - Initializes `sample_data` instances for all samples in a specified input folder.
    - The class automatically identifies and matches files by sample name for assembly files, Illumina read files, and Nanopore read files.

3. **Metadata Sheet-Based Initialization**:
    - Initializes `sample_data` instances for all samples listed in a provided metadata sheet.
    - This metadata sheet should contain paths to each sample’s relevant files.

For all three options, additional metadata can be provided in TSV format and will be included in the `metadata` attribute of each `sample_data` instance.

---


In [ ]:
# |export


class sample_data:
    """
    Class to handle sample data
    """

    species = "Unspecified"

    def __init__(self,
                 attributes,
                 input_folder):
        """
        Initialize the class 

        Parameters
        -------------
        attributes : dict 
            dictionary containing sample information 
        input_folder : string
            Path to input folder e.g reads
        """
        attributes = attributes.copy()
        #  Try to get the absolute path of the input folder
        try:
            self.input_folder = os.path.abspath(input_folder)
        except TypeError:
            # Set to None if there's an issue with input_folder
            self.input_folder = None 
        
        # Try to retrieve the sample name from attributes
        try:
            self.sample_name: str = attributes['sample_name']
        except KeyError:
            self.sample_name = "sample" 

        try:
            # Try to retrieve the assembly file path and make it absolute
            self.assembly_file = get_abs_file_path(self.input_folder,
                                                   attributes['assembly_file'])
        except KeyError:
            # If 'assembly_file' key doesn't exist
            self.assembly_file = None

        # Try to retrieve Illumina read file
        if "Illumina_read_files" in attributes and attributes["Illumina_read_files"] is not None:
            if isinstance(attributes['Illumina_read_files'], str):
                Illumina_read_files = attributes['Illumina_read_files'].split(",")
            elif isinstance(attributes['Illumina_read_files'], list):
                Illumina_read_files = attributes['Illumina_read_files']
            self.Illumina_read_files = [get_abs_file_path(self.input_folder,
                                                          Illumina_read_files[0]), # First read
                                        get_abs_file_path(self.input_folder,
                                                          Illumina_read_files[1])] # Second read
        else:
            self.Illumina_read_files = None # If 'Illumina_read_files' key doesn't exist

        try:
            # Try to retrieve Nanopore read file and make it absolute
            self.Nanopore_read_file: Path = get_abs_file_path(self.input_folder,
                                                              attributes['Nanopore_read_file'])
        except KeyError:
            self.Nanopore_read_file = None # If 'Nanopore_read_file' key doesn't exist
        # Store all attributes as metadata
        self.metadata: dict = attributes

In [ ]:
# |export

class input_manager:
    """
    class to manage input configurations and sample initialization
    """
    
    
    def __init__(self,
                 input_config):
        """
        Initialize the class 

        Parameters
        -------------
        input_config : yaml file
            Configuration file with options to handle samples
        """ 
        

        try:
            # Try to set the base output folder; default to './' if not provided
            self.base_input_folder: Path = os.path.abspath(input_config["input_folder"])
        except:
            self.base_input_folder: Path = os.path.abspath("./")
        self.get_samplesheet(input_config)

        # Decide on loading method based on configuration settings
        if input_config["load_from_samplesheet"]:
            # If loading from samplesheet, update base input folder if not specified
            if not input_config["input_folder"] or input_config["input_folder"] is None:
                self.base_input_folder = os.path.dirname(self.metadata_file)
            self.add_samples_from_samplesheet()
        else:
            # Load samples directly from specified folder
            self.add_samples_from_folder(input_config)
    
    ### TODO: figure out how to deal with duplicate sample names. Just a warning?

    def get_samplesheet(self,
                        input_config):
        """
        Method to retrieve the samplesheet from the configuration file
        and use to update metadata

        Parameters
        -------------
        input_config : yaml file
            Configuration file with options to handle samples
        """
        # Check if there is a samplesheet provided in the configuration
        if "samplesheet" in input_config and input_config["samplesheet"] is not None:
            self.metadata_file = os.path.abspath(input_config["samplesheet"])
            self.metadata = self.get_metadata_from_samplesheet() # Get metadata
        else:
            # No information
            self.metadata_file = None
            self.metadata = None


    def init_sample(self,
                    attributes):
        """
        Method to initialize a single sample object from sample_data class

        Parameters
        -------------
        attributes : dict 
            dictionary containing sample information 
        """
        
        sample = sample_data(attributes,
                             self.base_input_folder)
        return(sample)
              
    def add_samples(self,
                    file_paths):
        """
        Method to initialize multiple sample objects based on file paths

        Parameters
        -------------
        file_paths : dict 
            dictionary containing sample information 
        """  

        # Create a list to hold the sample objects
        self.samples = []
        for sample_name in file_paths:
            attributes = file_paths[sample_name].copy()
            attributes["sample_name"] = sample_name # Assign sample name to the attributes
            self.samples.append(self.init_sample(attributes)) # Add the sample to the list
        return(file_paths)

    def add_samples_from_samplesheet(self):
        # Load file paths from samplesheet
        file_paths = self.get_metadata_from_samplesheet()
        # Add samples to samples list based on these paths
        self.add_samples(file_paths)
    
    def add_samples_from_folder(self,
                                input_config):
        # Load file paths from specified folder
        file_paths = self.get_input_from_folder(input_config)
        # Add samples to samples list based on these paths
        self.add_samples(file_paths)

    def get_input_from_folder(self,
                              input_config):
        """
        Method to retrieve input files from a folder and classify them by type (e.g., Illumina or Nanopore) 

        Parameters
        -------------
        input_config : yaml file
            Configuration file with options to handle samples
        """ 
        # Regex for Illumina reads
        illumina_regex = re.compile(input_config["file_patterns"]["Illumina_read_file"])
        # Regex for Nanopore reads
        nanopore_regex = re.compile(input_config["file_patterns"]["Nanopore_read_file"])
        # Regex for assembly files
        assembly_regex = re.compile(input_config["file_patterns"]["assembly_fasta_file"])
        
        
        input_folder = self.base_input_folder
        file_paths = {} # Dict to store file paths by sample
        files = os.listdir(input_folder)
        
        # Iterate over files
        for file in files:
            sample_name = False
            if file.endswith((".fa",".fasta",".fna")): # Assembly data
                # Extract sample name from filename
                sample_name = re.match(assembly_regex,
                                       file).group("sample_name")
                if sample_name in file_paths:
                    file_paths[sample_name]["assembly_file"] = file
                else:
                    file_paths[sample_name] = {"assembly_file": file}
            
            # Reads
            elif file.endswith(".fastq.gz"):
                re_match = re.match(illumina_regex,file)
                if re_match:
                    # Get sample name
                    sample_name = re_match.group("sample_name")
                    # Get read number
                    read_number = re_match.group("paired_read_number")
                    
                    # Distinguish between R1 and R2
                    if read_number == "R1":  # Read 1
                        if sample_name in file_paths:
                            if "Illumina_read_files" in file_paths[sample_name]:
                                file_paths[sample_name]["Illumina_read_files"] = file + "," + file_paths[sample_name]["Illumina_read_files"]
                            else:
                                file_paths[sample_name]["Illumina_read_files"] = file
                        else:
                            file_paths[sample_name] = {"Illumina_read_files": file}
                    elif read_number == "R2": # Read 2
                        if sample_name in file_paths:
                            if "Illumina_read_files" in file_paths[sample_name]:
                                file_paths[sample_name]["Illumina_read_files"] = file_paths[sample_name]["Illumina_read_files"] + "," + file
                            else:
                                file_paths[sample_name]["Illumina_read_files"] = file
                        else:
                            file_paths[sample_name] = {"Illumina_read_files": file}
                
                #### TODO: Logic for nanopore read data, need file naming conventions
                
                # Nanopore reads if present
                else:
                    re_match = re.match(nanopore_regex,file)
                    if re_match:
                        sample_name = re_match.group("sample_name")
                        if sample_name in file_paths:
                            file_paths[sample_name]["Nanopore_read_file"] = file
                        else:
                            file_paths[sample_name] = {"Nanopore_read_file": file}
        
        # Update attributes with metadata from samplesheet if provided
        if self.metadata_file is not None:
            for sample_name in file_paths:
                if sample_name in self.metadata:
                    file_paths_updated = self.metadata[sample_name].copy()
                    file_paths_updated.update(file_paths[sample_name])
                    file_paths[sample_name] = file_paths_updated
        return(file_paths)
    
    def get_metadata_from_samplesheet(self):
        # Load the samplesheet in a dataframe
        df = core.get_samplesheet({"path": self.metadata_file})
        df.set_index('sample_name', inplace=True)
        df_dict = df.to_dict("index")
        return(df_dict)

    def __iter__(self):
        """
        Method to iterate over the samples in the input_manager object
        """
        for sample in self.samples:
            yield(sample)

    def __len__(self):
        """
        Method that returns the number of samples managed by this object
        """
        return(len(self.samples))


In [ ]:
# |export

def get_abs_file_path(input_folder,
                      file_path):
    """
    Function to return the absolute path of a file
    """
    if file_path is not None:
        if input_folder:
            file_path = os.path.abspath(os.path.join(input_folder,file_path))
        else:
            file_path = os.path.abspath(file_path)
        if not os.path.exists(file_path):
            print(f"WARNING: {file_path} does not exist")
    return(file_path)


def get_single_sample_attributes(input_config):
    """
    Function to retrieve input information for a single sample.

    Parameters
    -------------
    input_config : yaml file
        Configuration file with options to handle samples
    """ 

    # Get file paths from input config
    file_paths = dict((k, input_config[k]) for k in ["sample_name",
                                                     "assembly_file",
                                                     "Illumina_read_files",
                                                     "Nanopore_read_file"] if k in input_config)
    # If the sample name is not found
    if not "sample_name" in input_config or input_config["sample_name"] is None:
        # If the assembly file is available
        if "assembly_file" in input_config and input_config["assembly_file"] is not None:
            # Add the sample name to the file path dictionary from the assembly file 
            file_paths["sample_name"] = re.match(input_config["file_patterns"]["assembly_fasta_file"],
                                                 os.path.basename(input_config["assembly_file"])).group("sample_name")
        # Otherwise take it from the Illumina reads
        elif "Illumina_read_files" in input_config and input_config["Illumina_read_files"] is not None:
            file_paths["sample_name"] = re.match(input_config["file_patterns"]["Illumina_read_file"],
                                                 os.path.basename(input_config["Illumina_read_files"].split(',')[0])).group("sample_name")
        # Otherwise take it from the Nanopore reads
        elif "Nanopore_read_file" in input_config and input_config["Nanopore_read_file"] is not None:
            file_paths["sample_name"] = re.match(input_config["file_patterns"]["assembly_fasta_file"],
                                                 os.path.basename(input_config["Nanopore_read_file"])).group("sample_name")
    
    # If a samplesheet is available in the config
    if input_config["samplesheet"] is not None:
        # Load sample sheet as DataFrame
        df = core.get_samplesheet({"path": input_config["samplesheet"]})
        try:
            # Set sample_name as the index if it exists
            df.set_index('sample_name', inplace=True)
            df_dict = df.to_dict("index") # Save the sample bane
            # Retrieve metadata for the specified sample name, if available
            if input_config["sample_name"] is not None and input_config["sample_name"] in df_dict:
                attributes = df_dict[input_config["sample_name"]]
            else:
                # Otherwise, use the first row in the sample sheet as default metadata
                attributes = df_dict[list(df_dict)[0]]
                print("Sample name not provided or not found in metadata. Using first row as metadata input")
        except KeyError:
            # If sample_name column is not found, fallback to using the first row of the DataFrame
            df_list = df.to_dict("records")
            attributes = df_list[0]
            print("No sample_name column in metadata. Using first row as metadata input")
    else:
        attributes = {}
    # Update attributes with file paths and sample name information
    attributes.update(file_paths)
    return(attributes)

In [ ]:
# |export
config = core.get_config("/dpssi/home/scrsim/ssi_analysis_utility/ssi_analysis_utility/config/config.default.yaml")
simone_testing = input_manager(config["input_manager"])
print(simone_testing.__dict__)


Python-dotenv could not parse statement starting at line 9
Python-dotenv could not parse statement starting at line 10
Python-dotenv could not parse statement starting at line 11
Python-dotenv could not parse statement starting at line 12
Python-dotenv could not parse statement starting at line 13
Python-dotenv could not parse statement starting at line 16
Python-dotenv could not parse statement starting at line 17
Python-dotenv could not parse statement starting at line 18
Python-dotenv could not parse statement starting at line 19
Python-dotenv could not parse statement starting at line 22
Python-dotenv could not parse statement starting at line 23
Python-dotenv could not parse statement starting at line 24
Python-dotenv could not parse statement starting at line 25
Python-dotenv could not parse statement starting at line 26
Python-dotenv could not parse statement starting at line 27
Python-dotenv could not parse statement starting at line 28
Python-dotenv could not parse statement s

{'file_patterns': {'Illumina_read_file': '(?P<sample_name>.+?)(?P<sample_number>(_S[0-9]+)?)(?P<lane>(_L[0-9]+)?)_(?P<paired_read_number>R[1|2])(?P<set_number>(_[0-9]+)?)(?P<file_extension>\\.fastq\\.gz)', 'Nanopore_read_file': '(?P<sample_name>.+?)_nanopore\\.fastq\\.gz', 'assembly_fasta_file': '(?P<sample_name>.+?)(\\.fa|\\.fna|\\.fasta)', 'metadata': 'r"(?P<metadata_name>.+?)(\\.tsv|\\.tab\\.txt)"'}, 'load_from_samplesheet': True, 'load_from_folder': False, 'input_folder': None, 'output_folder': '.', 'assembly_file': None, 'Illumina_read_files': None, 'Nanopore_read_file': None, 'samplesheet': '/dpssi/home/scrsim/ssi_analysis_utility/examples/samplesheet.tsv', 'sample_name': None}
{'base_input_folder': '/dpssi/home/scrsim/ssi_analysis_utility/examples', 'metadata_file': '/dpssi/home/scrsim/ssi_analysis_utility/examples/samplesheet.tsv', 'metadata': {'GAS-2022-1029': {'Illumina_read_files': 'GAS-2022-1029_S42_L555_R1_001.fastq.gz,GAS-2022-1029_S42_L555_R2_001.fastq.gz', 'Nanopore_rea

### Testing



In [ ]:
# |export

def single_test():
    example_sample = sample_data({"sample_name":"GAS-2022-1029",
                                       "assembly_file":"examples/GAS-2022-1029.fasta",
                                       "Illumina_read_files":["examples/GAS-2022-1029_S42_L555_R1_001.fastq.gz","examples/GAS-2022-1029_S42_L555_R2_001.fastq.gz"]},
                                       input_folder = False,
                                       )
    assert(len(example_sample.Illumina_read_files) == 2)
    assert(not example_sample.Nanopore_read_file)
    print(example_sample.__dict__)

def single_test_assembly():
    example_sample = sample_data({"assembly_file":"examples/GAS-2022-1029.fasta"},
                                       input_folder = False,
                                       )
    assert(not(example_sample.Illumina_read_files))
    assert(not example_sample.Nanopore_read_file)
    print(example_sample.__dict__)

def single_test_input_folder():
    config = core.get_config()
    example_sample = sample_data({"sample_name":"GAS-2024-0773",
                                       "assembly_file":"GAS-2024-0773.fasta",
                                       "Illumina_read_files":["GAS-2024-0773_S35_L555_R1_001.fastq.gz","GAS-2024-0773_S35_L555_R2_001.fastq.gz"]},
                                       input_folder = "examples",
                                       )
    assert(example_sample.sample_name == "GAS-2024-0773")
    assert(len(example_sample.Illumina_read_files) == 2)
    assert(not example_sample.Nanopore_read_file)
    print(example_sample.__dict__)


def test_folder():
    config = core.get_config()
    input_config =  config["input_manager"]
    input_config["load_from_folder"] = True
    input_config["input_folder"] = "examples/"
    test = input_manager(input_config)
    for x in test:
        print(x.__dict__)

def test_folder_with_metadata():
    config = core.get_config()
    input_config =  config["input_manager"]
    input_config["load_from_folder"] = True
    input_config["input_folder"] = "examples/"
    input_config["samplesheet"] = "examples/samplesheet.tsv"
    test = input_manager(input_config)
    for x in test:
        print(x.__dict__)

def samplesheet_test():
    config = core.get_config()
    print(config)
    input_config =  config["input_manager"]
    input_config["load_from_samplesheet"] = True
    input_config["samplesheet"] = "/dpssi/home/scrsim/ssi_analysis_utility/examples/samplesheet.tsv"
    test = input_manager(input_config)
    print(test.__dict__)

    # for x in test.__dict__:
    #     print(x)
    #     print(x.metadata["Illumina_read_files"])
    #     print(x.metadata["assembly_file"])
    #     print(x.metadata["sample_name"])
    #     print(x.assembly_file)
    #     print(x.Illumina_read_files)



In [ ]:

single_test()
single_test_assembly()
single_test_input_folder()


{'input_folder': None, 'sample_name': 'GAS-2022-1029', 'assembly_file': '/dpssi/home/scrsim/ssi_analysis_utility/examples/GAS-2022-1029.fasta', 'Illumina_read_files': ['/dpssi/home/scrsim/ssi_analysis_utility/examples/GAS-2022-1029_S42_L555_R1_001.fastq.gz', '/dpssi/home/scrsim/ssi_analysis_utility/examples/GAS-2022-1029_S42_L555_R2_001.fastq.gz'], 'Nanopore_read_file': None, 'metadata': {'sample_name': 'GAS-2022-1029', 'assembly_file': 'examples/GAS-2022-1029.fasta', 'Illumina_read_files': ['examples/GAS-2022-1029_S42_L555_R1_001.fastq.gz', 'examples/GAS-2022-1029_S42_L555_R2_001.fastq.gz']}}
{'input_folder': None, 'sample_name': 'sample', 'assembly_file': '/dpssi/home/scrsim/ssi_analysis_utility/examples/GAS-2022-1029.fasta', 'Illumina_read_files': None, 'Nanopore_read_file': None, 'metadata': {'assembly_file': 'examples/GAS-2022-1029.fasta'}}
{'input_folder': '/dpssi/home/scrsim/ssi_analysis_utility/examples', 'sample_name': 'GAS-2024-0773', 'assembly_file': '/dpssi/home/scrsim/ssi_

In [ ]:
test_folder()

{'input_folder': '/dpssi/home/scrsim/ssi_analysis_utility/examples', 'sample_name': 'GAS-2023-0253', 'assembly_file': '/dpssi/home/scrsim/ssi_analysis_utility/examples/GAS-2023-0253.fasta', 'Illumina_read_files': ['/dpssi/home/scrsim/ssi_analysis_utility/examples/GAS-2023-0253_S74_L555_R1_001.fastq.gz', '/dpssi/home/scrsim/ssi_analysis_utility/examples/GAS-2023-0253_S74_L555_R2_001.fastq.gz'], 'Nanopore_read_file': '/dpssi/home/scrsim/ssi_analysis_utility/examples/GAS-2023-0253_nanopore.fastq.gz', 'metadata': {'Nanopore_read_file': 'GAS-2023-0253_nanopore.fastq.gz', 'assembly_file': 'GAS-2023-0253.fasta', 'Illumina_read_files': 'GAS-2023-0253_S74_L555_R1_001.fastq.gz,GAS-2023-0253_S74_L555_R2_001.fastq.gz', 'sample_name': 'GAS-2023-0253'}}
{'input_folder': '/dpssi/home/scrsim/ssi_analysis_utility/examples', 'sample_name': 'GAS-2022-1029', 'assembly_file': '/dpssi/home/scrsim/ssi_analysis_utility/examples/GAS-2022-1029.fasta', 'Illumina_read_files': ['/dpssi/home/scrsim/ssi_analysis_uti

In [ ]:
test_folder_with_metadata()

{'input_folder': '/dpssi/home/scrsim/ssi_analysis_utility/examples', 'sample_name': 'GAS-2023-0253', 'assembly_file': '/dpssi/home/scrsim/ssi_analysis_utility/examples/GAS-2023-0253.fasta', 'Illumina_read_files': ['/dpssi/home/scrsim/ssi_analysis_utility/examples/GAS-2023-0253_S74_L555_R1_001.fastq.gz', '/dpssi/home/scrsim/ssi_analysis_utility/examples/GAS-2023-0253_S74_L555_R2_001.fastq.gz'], 'Nanopore_read_file': '/dpssi/home/scrsim/ssi_analysis_utility/examples/GAS-2023-0253_nanopore.fastq.gz', 'metadata': {'Illumina_read_files': 'GAS-2023-0253_S74_L555_R1_001.fastq.gz,GAS-2023-0253_S74_L555_R2_001.fastq.gz', 'Nanopore_read_file': 'GAS-2023-0253_nanopore.fastq.gz', 'assembly_file': 'GAS-2023-0253.fasta', 'organism': 'Streptococcus pyogenes', 'variant': 'M1DK', 'notes': 'Nan', 'sample_name': 'GAS-2023-0253'}}
{'input_folder': '/dpssi/home/scrsim/ssi_analysis_utility/examples', 'sample_name': 'GAS-2022-1029', 'assembly_file': '/dpssi/home/scrsim/ssi_analysis_utility/examples/GAS-2022-

In [ ]:
samplesheet_test()

{'SHELL': '/bin/bash', 'HISTCONTROL': 'erasedups', 'CONDA_EXE': '/users/data/Tools/Conda/Miniconda3-py311_23.5.2-0-Linux-x86_64/bin/conda', '_CE_M': '', 'HISTSIZE': '10000', 'HISTTIMEFORMAT': '[%F %T] ', 'XML_CATALOG_FILES': 'file:///dpssi/home/scrsim/conda_envs/.venv/etc/xml/catalog file:///etc/xml/catalog', 'ENV': '/cm/local/apps/environment-modules/4.2.1//init/profile.sh', 'PWD': '/dpssi/home/scrsim', 'KRB5CCNAME': 'KCM:', 'LOGNAME': 'scrsim', 'XDG_SESSION_TYPE': 'tty', 'CONDA_PREFIX': '/dpssi/home/scrsim/conda_envs/.venv', 'MODULESHOME': '/cm/local/apps/environment-modules/4.2.1/', 'MOTD_SHOWN': 'pam', 'HOME': '/dpssi/home/scrsim', 'LANG': 'en_US.UTF-8', 'CONDA_PROMPT_MODIFIER': '(/dpssi/home/scrsim/conda_envs/.venv) ', 'VSCODE_AGENT_FOLDER': '/dpssi/home/scrsim/.vscode-server', 'SSH_CONNECTION': '10.96.143.48 50049 10.45.128.75 22', 'XDG_SESSION_CLASS': 'user', '_CE_CONDA': '', 'LESSOPEN': '||/usr/bin/lesspipe.sh %s', 'USER': 'scrsim', 'CONDA_SHLVL': '2', 'VISUAL': 'nano', 'LOADED

In [ ]:
# | hide
# This is included at the end to ensure when you run through your notebook the code is also transferred to the module and isn't just a notebook
import nbdev

nbdev.nbdev_export()